In [2]:
import pandas as pd
import numpy as np
import sys

# retrieving utils from parent directory
sys.path.append('..')
from utils.utils import *  # Import all functions from utils.utils

utils module initialized


#### Loading the dataset
---

In [3]:
csv_path = '../data/hospital_general_info.csv'

df = pd.read_csv(csv_path)

#### Initial Exploratory Data Analysis (EDA)
---

In [4]:
from utils.utils import basic_eda
# Perform basic EDA
basic_eda(df)

First 5 rows:
  Facility ID                    Facility Name                     Address  \
0       10001  SOUTHEAST HEALTH MEDICAL CENTER      1108 ROSS CLARK CIRCLE   
1       10005         MARSHALL MEDICAL CENTERS  2505 U S HIGHWAY 431 NORTH   
2       10006     NORTH ALABAMA MEDICAL CENTER         1701 VETERANS DRIVE   
3       10007         MIZELL MEMORIAL HOSPITAL               702 N MAIN ST   
4       10008      CRENSHAW COMMUNITY HOSPITAL         101 HOSPITAL CIRCLE   

  City/Town State  ZIP Code County/Parish Telephone Number  \
0    DOTHAN    AL     36301       HOUSTON   (334) 793-8701   
1      BOAZ    AL     35957      MARSHALL   (256) 593-8310   
2  FLORENCE    AL     35630    LAUDERDALE   (256) 768-8400   
3       OPP    AL     36467     COVINGTON   (334) 493-3541   
4   LUVERNE    AL     36049      CRENSHAW   (334) 335-3374   

          Hospital Type                           Hospital Ownership  ...  \
0  Acute Care Hospitals  Government - Hospital District or Authorit

#### Cleaning the data
---

In [5]:
#Standardize the column names for consistency
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
df.columns = df.columns.str.replace('/', '_')

print("Standardized column names:", df.columns.tolist())

Standardized column names: ['facility_id', 'facility_name', 'address', 'city_town', 'state', 'zip_code', 'county_parish', 'telephone_number', 'hospital_type', 'hospital_ownership', 'emergency_services', 'meets_criteria_for_birthing_friendly_designation', 'hospital_overall_rating', 'hospital_overall_rating_footnote', 'mort_group_measure_count', 'count_of_facility_mort_measures', 'count_of_mort_measures_better', 'count_of_mort_measures_no_different', 'count_of_mort_measures_worse', 'mort_group_footnote', 'safety_group_measure_count', 'count_of_facility_safety_measures', 'count_of_safety_measures_better', 'count_of_safety_measures_no_different', 'count_of_safety_measures_worse', 'safety_group_footnote', 'readm_group_measure_count', 'count_of_facility_readm_measures', 'count_of_readm_measures_better', 'count_of_readm_measures_no_different', 'count_of_readm_measures_worse', 'readm_group_footnote', 'pt_exp_group_measure_count', 'count_of_facility_pt_exp_measures', 'pt_exp_group_footnote', 't

In [6]:
#drop columns that are not needed
columns_to_drop = ['telephone_number', 'county_parish', 'address',
    'meets_criteria_for_birthing_friendly_designation',
    'mort_group_measure_count',
    'count_of_facility_mort_measures',
    'count_of_mort_measures_no_different',
    'count_of_mort_measures_worse',
    'mort_group_footnote',
    'safety_group_measure_count',
    'count_of_facility_safety_measures',
    'count_of_safety_measures_no_different',
    'count_of_safety_measures_worse',
    'safety_group_footnote',
    'readm_group_measure_count',
    'count_of_facility_readm_measures',
    'count_of_readm_measures_no_different',
    'count_of_readm_measures_worse',
    'readm_group_footnote',
    'pt_exp_group_measure_count',
    'count_of_facility_pt_exp_measures',
    'pt_exp_group_footnote',
    'te_group_measure_count',
    'count_of_facility_te_measures',
    'te_group_footnote',
    'emergency_services',
    'count_of_mort_measures_better',
    'count_of_safety_measures_better',
    'count_of_readm_measures_better'
    ]

df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
print("Remaining columns after dropping:", df.columns.tolist())


Remaining columns after dropping: ['facility_id', 'facility_name', 'city_town', 'state', 'zip_code', 'hospital_type', 'hospital_ownership', 'hospital_overall_rating', 'hospital_overall_rating_footnote']


#### Mapping Coded Values to Footnote Crosswalk

---

##### I will use the CMS data dictionary footnote crosswalk to map coded values in the hospital_overall_rating_footnote column to their corresponding footnotes. This will help in understanding the null values in the hospital_overall_rating column.

In [7]:
# FOOTNOTE CROSSWALK
footnote_mapping: dict[int, str] = {
    1: "The number of cases/patients is too few to report.",
    2: "Data submitted were based on a sample of cases/patients.",
    3: "Results are based on a shorter time period than required.",
    4: "Data suppressed by CMS for one or more quarters.",
    5: "Results are not available for this reporting period.",
    6: "Fewer than 100 patients completed the HCAHPS survey.",
    7: "No cases met the criteria for this measure.",
    8: "Lower limit of CI cannot be calculated if observed infections = 0.",
    9: "No data available from the state/territory for this reporting period.",
    10: "Very few patients completed HCAHPS; fewer than 50 surveys.",
    11: "Discrepancies in the data collection process.",
    12: "This measure does not apply to this hospital for this reporting period.",
    13: "Results cannot be calculated for this reporting period.",
    14: "Results combined with nearby states to protect confidentiality.",
    15: "Too few patients to report a star rating.",
    16: "Too few measures or groups reported to calculate a star rating.",
    17: "Star rating includes only inpatient services.",
    18: "Result not based on performance; no data submitted, no exemption.",
    19: "Data shown only for hospitals in IQR/OQR programs.",
    20: "State/national averages exclude VHA hospital data.",
    21: "VHA survey results excluded from official HCAHPS and averages.",
    22: "No overall star rating for DoD hospitals.",
    23: "Discrepancies in hospital/facility claims data.",
    24: "VA hospital results combined with VA administrative parent.",
    25: "State/national averages include VHA data.",
    26: "State/national averages include DoD data.",
    27: "DoD survey results excluded from official HCAHPS and averages.",
    28: "CMS-approved Exception may impact results.",
    29: "Partial performance period due to CMS-approved Exception."
}

 ##### Creating new column with footnote label which contains description of each footnote value.

In [8]:
#Footnote label column
df['footnote_label'] = pd.to_numeric(df['hospital_overall_rating_footnote'], errors='coerce').map(footnote_mapping)

#print hospital_overall_rating_footnote and footnote_label columns
print(df[['hospital_overall_rating','hospital_overall_rating_footnote', 'footnote_label']].head(5))

  hospital_overall_rating hospital_overall_rating_footnote  \
0                       3                              NaN   
1                       2                              NaN   
2                       1                              NaN   
3                       1                              NaN   
4           Not Available                               16   

                                      footnote_label  
0                                                NaN  
1                                                NaN  
2                                                NaN  
3                                                NaN  
4  Too few measures or groups reported to calcula...  


In [ ]:
# Reviewing the data
print("Total null values for each column:")
print(df.isnull().sum())

# print sum of values "not applicable" in hospital_overall_rating_footnote column
print("Sum of 'not applicable' in hospital_overall_rating_footnote:", (df['hospital_overall_rating_footnote'] == 'not applicable').sum())

#print unique values in hospital_overall_rating_footnote and their corresponding footnote_label columns side by side
print("Unique values in hospital_overall_rating_footnote and their corresponding footnote_label:")
print(df[['hospital_overall_rating_footnote', 'footnote_label']].drop_duplicates().sort_values(by='hospital_overall_rating_footnote'))

Total null values for each column:
facility_id                            0
facility_name                          0
city_town                              0
state                                  0
zip_code                               0
hospital_type                          0
hospital_ownership                     0
hospital_overall_rating                0
hospital_overall_rating_footnote    2766
footnote_label                      2767
dtype: int64
Sum of 'not applicable' in hospital_overall_rating_footnote: 0
Unique values in hospital_overall_rating_footnote and their corresponding footnote_label:
     hospital_overall_rating_footnote  \
4                                  16   
3088                           16, 23   
347                                17   
88                                 19   
108                                22   
817                                23   
188                                 5   
0                                 NaN   

                   

In [ ]:
# create function to handle multiple values in hospital_overall_rating_footnote
def parse_footnote(footnote):
    if pd.isnull(footnote):
        return None
    footnote = str(footnote).strip('()')
    return tuple(sorted(map(int, footnote.split(',')))) if ',' in footnote else int(footnote)
# Apply the function to the hospital_overall_rating_footnote column
df['hospital_overall_rating_footnote'] = df['hospital_overall_rating_footnote'].apply(parse_footnote)
# Display the updated DataFrame
print("Updated hospital_overall_rating_footnote with parsed values:")
# print all unique values in the hospital_overall_rating_footnote column
print(df['hospital_overall_rating_footnote'].unique())
df['hospital_overall_rating_footnote'] = df['hospital_overall_rating_footnote'].apply(lambda x: x if isinstance(x, tuple) else (x,))

Updated hospital_overall_rating_footnote with parsed values:
[None 16 19 22 5 17 23 (16, 23)]


### Null Handling Strategy:
#### The unique values in the `hospital_overall_rating` indicate there are missing footnote values of 5, 16, 19, 22, (16, 23).  I will handle these null values as follows:
1. If the `hospital_overall_rating` value is null, I will check the corresponding `hospital_overall_rating_footnote` for a footnote value.
2. If the footnote value is 5, I will drop these rows as they are not useful for analysis.
3. If the footnote value is 16, I will drop these rows as they are not useful for analysis.
4. If footnote values is 19, I will drop these rows as they are not useful for analysis.
5. If the footnote value is 22, I will drop these rows as they are not useful for analysis.
6. If the footnote value is 23, I will drop these rows as they may not accurately reflect the hospital's performance.

#### The footnote value of 17 indicates that the hospital star rating is only based on inpatient data.  I will keep these rows as they may still provide useful information about the hospital's performance.

------------------------------------------------
#### Summary Strategy


| Footnote Value | Description                                                                 | Action                                      |
|----------------|-----------------------------------------------------------------------------|---------------------------------------------|
| 5              | Results are not available for this reporting period.                        | Drop rows                                   |
| 16             | Too few measures or groups reported to calculate a star rating.             | Drop rows                                   |
| 17             | Star rating is based on inpatient data only.                                | Keep rows                                   |
| 19             | Data shown only for hospitals in IQR/OQR programs.                          | Drop rows                                   |
| 22             | No overall star rating for Department of Defense hospitals.                 | Drop rows                                   |
| 23             | The hospital or facility has reported discrepancies in their claims data. | Drop rows                                   |

In [ ]:
# Drop rows where hospital_overall_rating_footnote contains ONLY 5, 16, 19, 22, or (16, 23)
drop_footnotes = {(5,), (16,), (19,), (22,), (16, 23)}
def should_drop(footnote):
	# If not a tuple, make it a tuple for comparison
	if isinstance(footnote, tuple):
		return tuple(sorted(footnote)) in drop_footnotes
	try:
		return (int(footnote),) in drop_footnotes
	except Exception:
		return False

df = df[~df['hospital_overall_rating_footnote'].apply(should_drop)]
print("Unique values in hospital_overall_rating_footnote after filtering:", df['hospital_overall_rating_footnote'].unique())

Unique values in hospital_overall_rating_footnote after filtering: [(None,) (17,) (23,)]


##### A footnote label will not be present when hospital_overall_rating_footnote is 'not applicable'. If the null footnote_label values are equal to hospital_overall_rating_footnote, I will also fill these with 'not applicable'.

In [ ]:
#Reviewing the cleaned data
print("Cleaned Data:")
print(df.head())

Cleaned Data:
  facility_id                    facility_name   city_town state  zip_code  \
0       10001  SOUTHEAST HEALTH MEDICAL CENTER      DOTHAN    AL     36301   
1       10005         MARSHALL MEDICAL CENTERS        BOAZ    AL     35957   
2       10006     NORTH ALABAMA MEDICAL CENTER    FLORENCE    AL     35630   
3       10007         MIZELL MEMORIAL HOSPITAL         OPP    AL     36467   
5       10011               ST. VINCENT'S EAST  BIRMINGHAM    AL     35235   

          hospital_type                           hospital_ownership  \
0  Acute Care Hospitals  Government - Hospital District or Authority   
1  Acute Care Hospitals  Government - Hospital District or Authority   
2  Acute Care Hospitals                                  Proprietary   
3  Acute Care Hospitals               Voluntary non-profit - Private   
5  Acute Care Hospitals               Voluntary non-profit - Private   

  hospital_overall_rating hospital_overall_rating_footnote footnote_label  
0       

In [ ]:
df_index = df.index
df

,facility_id,facility_name,city_town,state,zip_code,hospital_type,hospital_ownership,hospital_overall_rating,hospital_overall_rating_footnote,footnote_label
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,DOTHAN,AL,36301,Acute Care Hospitals,Government - Hospital District or Authority,3,"(None,)",NaN
1,10005,MARSHALL MEDICAL CENTERS,BOAZ,AL,35957,Acute Care Hospitals,Government - Hospital District or Authority,2,"(None,)",NaN
2,10006,NORTH ALABAMA MEDICAL CENTER,FLORENCE,AL,35630,Acute Care Hospitals,Proprietary,1,"(None,)",NaN
3,10007,MIZELL MEMORIAL HOSPITAL,OPP,AL,36467,Acute Care Hospitals,Voluntary non-profit - Private,1,"(None,)",NaN
5,10011,ST. VINCENT'S EAST,BIRMINGHAM,AL,35235,Acute Care Hospitals,Voluntary non-profit - Private,3,"(None,)",NaN
...,...,...,...,...,...,...,...,...,...,...
5355,670120,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,EL PASO,TX,79911,Acute Care Hospitals,Proprietary,1,"(None,)",NaN
5357,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,THE WOODLANDS,TX,77385,Acute Care Hospitals,Voluntary non-profit - Private,5,"(None,)",NaN
5368,670260,TEXAS HEALTH HOSPITAL FRISCO,FRISCO,TX,75033,Acute Care Hospitals,Voluntary non-profit - Private,3,"(None,)",NaN
5374,670300,METHODIST MIDLOTHIAN MEDICAL CENTER,MIDLOTHIAN,TX,76065,Acute Care Hospitals,Voluntary non-profit - Private,4,"(None,)",NaN


In [ ]:
import os

#save the cleaned DataFrame to a new CSV file
clean_data_dir = '../data/clean_data'
os.makedirs(clean_data_dir, exist_ok=True)
cleaned_csv_path = os.path.join(clean_data_dir, 'cleaned_general_info.csv')
df.to_csv(cleaned_csv_path, index=False, encoding='utf-8-sig')  # BOM for pandas compatibility
print(f"Cleaned data saved to {cleaned_csv_path}")

Cleaned data saved to ../data/clean_data\cleaned_general_info.csv
